In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [2]:
test_ext = pd.read_csv('../../data/games/test_ext.csv')  
print(test_ext.shape)

test_ext.drop(columns=['author_id'], inplace=True)


(60346, 17)


In [3]:
test_ext['text_len'] = test_ext['content'].str.len()
test_ext['log_text_len'] = np.log1p(test_ext['text_len'])

bins = [0,20,100,500,np.inf]
labels = ['s', 'm', 'l', 'xl']
test_ext['len_bin'] = pd.cut(
    test_ext['text_len'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

test_ext = test_ext.drop(columns=['text_len'])
test_ext['len_bin'] = test_ext['len_bin'].astype('category')

In [4]:
test_ext = test_ext.drop(columns=['price_final'])
test_ext = test_ext.drop(columns=['review_score'])
test_ext = test_ext.drop(columns=['discount_percent'])
test_ext = test_ext.drop(columns=['log_text_len'])

In [5]:
test_ext['total'] = test_ext['positive'] + test_ext['negative']
test_ext['pos_ratio'] = test_ext['positive'] / test_ext['total']
test_ext = test_ext.drop(columns=['positive',
                                    'negative', 'steamspy_positive',
                                    'steamspy_negative'])



In [6]:
labels = ['early','mid-era','recent','latest']
test_ext['year_bin'] = pd.qcut(
    test_ext['release_year'],
    q=4,
    labels=labels
)

In [7]:
test_ext = test_ext.drop(columns=['release_year'])
test_ext['year_bin'] = test_ext['year_bin'].astype('category')

In [8]:
test_ext['log_recommendations'] = np.log1p(test_ext['recommendations'])
test_ext = test_ext.drop(columns=['recommendations'])

In [9]:
test_ext = test_ext.drop(columns=['app_id', 'price_initial', 'total'])

In [10]:
drop_cols = ['name', 'currency']
X_test = test_ext.drop(columns=drop_cols)
X_test.info()

num_feats=['is_free', 'price', 'log_recommendations', 'pos_ratio']
cat_feats=['len_bin', 'year_bin']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60346 entries, 0 to 60345
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   content              60346 non-null  object  
 1   is_free              60346 non-null  int64   
 2   price                60346 non-null  float64 
 3   len_bin              60346 non-null  category
 4   pos_ratio            60346 non-null  float64 
 5   year_bin             60346 non-null  category
 6   log_recommendations  60346 non-null  float64 
dtypes: category(2), float64(3), int64(1), object(1)
memory usage: 2.4+ MB


In [11]:
import joblib

print(test_ext.isna().sum())

clf = joblib.load('../../catboost_tfidf_pipeline.joblib')
preds = clf.predict_proba(X_test)[:, 1]
test_ext['sentiment'] = (preds >= .5).astype(int)


sub = test_ext[['sentiment']].copy()
sub.index = np.arange(1, len(sub) + 1)
sub.index.name = 'id'
sub.to_csv('./submission.csv')

content                0
name                   0
is_free                0
price                  0
currency               0
len_bin                0
pos_ratio              0
year_bin               0
log_recommendations    0
dtype: int64


In [12]:
test_ext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60346 entries, 0 to 60345
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   content              60346 non-null  object  
 1   name                 60346 non-null  object  
 2   is_free              60346 non-null  int64   
 3   price                60346 non-null  float64 
 4   currency             60346 non-null  object  
 5   len_bin              60346 non-null  category
 6   pos_ratio            60346 non-null  float64 
 7   year_bin             60346 non-null  category
 8   log_recommendations  60346 non-null  float64 
 9   sentiment            60346 non-null  int64   
dtypes: category(2), float64(3), int64(2), object(3)
memory usage: 3.8+ MB
